In [1]:
from module_Plot import Plotting
from module_Analyses_structure import Analyses as ana

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import ticker, cm, colors

import cartopy.crs as ccrs

import numpy as np
import pandas as pd
import vaex as ve
import os
from glob import glob

plt.rcParams['axes.unicode_minus'] = False


In [2]:
DIR_case = r'E:\data\_results\common_radar\case_sele'
fn_case = 'TRMMKu-93121_GPM-356_dT-3.8_BRpixel2715_grp64_p1..2014-03-22_13+12+42..2014-03-22_13+08+52..h5'
icase = os.path.join(DIR_case, fn_case)

vslices = [[126.6, 12.8], [129.9, 9.1]]
ll_resolution, slice_ymax = 1./30., 16



In [3]:
import netCDF4 as nc
terrain_file = r'D:\static_data\terrain\ETOPO2v2g_f4.nc'
terr = nc.Dataset(terrain_file)
terr.set_auto_mask(False) 

Lon_terr = terr.variables['x'][:]
Lat_terr = terr.variables['y'][:]
Terrain = terr.variables['z'][:,:].T / 1000  # 单位转为km
terr.close()

Lat_grid, Lon_grid = np.meshgrid(Lat_terr, Lon_terr)
Lon_grid.shape, Lat_grid.shape

((10801, 5401), (10801, 5401))

In [4]:
from pyresample import geometry, kd_tree
from scipy.spatial.distance import pdist, squareform
import h5py 

hgt = np.arange(0., 22., 0.125)[::-1].astype(np.float32)

#############
def gernerate_points(llpts, line_interp=True):
#    等间隔地生成二维直角坐标系下两点间连线上的点.包含端点两点共npts个点.
    dists = squareform(pdist(llpts))
    dists = dists[(np.arange(dists.shape[0]-1), np.arange(dists.shape[0]-1)+1)]
    if line_interp:
        xp, yp = np.empty(0), np.empty(0)
        for idx, p1 in enumerate(llpts[1:]):
            p0 = llpts[idx]
            npts = int(np.ceil(dists[idx]/ll_resolution))
            xtmp = np.linspace(p0[0], p1[0], npts)
            xp = np.hstack([xp, xtmp])                

            if np.isclose((p1[0] - p0[0]), 0):   # S-N direction
                yp = np.hstack([yp, np.linspace(p0[1], p1[1], npts)])
            else:
                k = (p1[1] - p0[1]) / (p1[0] - p0[0])
                if np.isclose(k, 0):   # W-E direction
                    yp = np.hstack([yp, np.full(npts, p1[1])])
                else:
                    b = p1[1] - k * p1[0]
                    yp = np.hstack([yp, k * xtmp + b])
    else:
        xp, yp = llpts[:,0], llpts[:,1]

    return xp, yp


def generat_slice(da3D, Lon1D, Lat1D, llpts_end, is_terr=True):
    xp, yp = gernerate_points(llpts_end)    
#    print('xp=',xp,'\nyp=', yp)
    swath_base = geometry.SwathDefinition(Lon1D, Lat1D)   
    swath_aim = geometry.SwathDefinition(xp, yp)
#    print(Lon1D.shape, da3D.shape)
    
#    valid_input_index, valid_output_index, index_array, distance_array = \
#                                kd_tree.get_neighbour_info(swath_base, swath_aim, 5000, neighbours=1)  
    Vslice = kd_tree.resample_nearest(swath_base, da3D, swath_aim, radius_of_influence=5000)
 
    if is_terr:
        grid_base = geometry.GridDefinition(lons=Lon_grid, lats=Lat_grid) 
        z_terrain = kd_tree.resample_nearest(grid_base, Terrain, swath_aim, radius_of_influence=5000)
    else:
        z_terrain = np.array(0)
    return (xp, yp), Vslice, z_terrain


In [5]:

def calc_cross_pts(VA, VB, rngs):
    Ks, Bs = [], []
    for La in VA:
        Ks.append( (La[1][0] - La[1][1])/(La[0][0] - La[0][1]) )
        Bs.append( La[1][0] - Ks[-1]*La[0][0] )

    for La in VB:
        Ks.append( (La[1][0] - La[1][1])/(La[0][0] - La[0][1]) )
        Bs.append( La[1][0] - Ks[-1]*La[0][0] )
    
    X, Y = [], []
    for idx in [0, 1]:
        for jdx in [2, 3]:
            X.append( -(Bs[idx] - Bs[jdx]) / (Ks[idx] - Ks[jdx]) )
            Y.append( Ks[idx]*X[-1]+Bs[idx]  )
    
    X, Y = np.array(X), np.array(Y)    
    res = [[X[0], X[1], X[-1], X[-2], X[0]],
           [Y[0], Y[1], Y[-1], Y[-2], Y[0]]]
    return res

##########
ANA_handler = ana(['','',''])

cases = {}
for idx, fn in enumerate([icase]):
    handle = h5py.File(fn, 'r')
    cases[idx] = {key: val[:] for key, val in handle.items()}
    cases[idx]['TRMM_Orbit'], cases[idx]['GPM_Orbit'] = cases[idx].pop('orbit')
    #cases[idx]['TRMM_Time'], cases[idx]['GPM_Time'] = fn.replace('+', ':').replace('_', ' ').split('..')[1:-1]
    
    cases[idx]['dT'] = fn.split('dT')[-1].split('_')[0]
    '''
    cases[idx]['levs_dBZ'], cases[idx]['unit_dBZ'] = ANA_handler.rainfall_levs('nearSurfdBZ'.lower(), 
               max(np.nanmax(cases[idx]['TRMM_nearSurfdBZ']), np.nanmax(cases[idx]['GPM_nearSurfdBZ'])))
    '''
    cases[idx]['levs_dBZ'], cases[idx]['unit_dBZ'] = np.arange(12,61,2), 'dBZ'           
    cases[idx]['levs_RR'], cases[idx]['unit_RR'] = ANA_handler.rainfall_levs('nearSurfRain'.lower(), 
               max(np.nanmax(cases[idx]['TRMM_nearSurfRain']), np.nanmax(cases[idx]['GPM_nearSurfRain'])))
    
    vertexsA = [[cases[idx]['X0'][[0, -1],0], cases[idx]['Y0'][[0, -1],0]],
                [cases[idx]['X0'][[0, -1],-1], cases[idx]['Y0'][[0, -1],-1]]]
    vertexsB = [[cases[idx]['X1'][[0, -1],0], cases[idx]['Y1'][[0, -1],0]],
                [cases[idx]['X1'][[0, -1],-1], cases[idx]['Y1'][[0, -1],-1]]]
    cases[idx]['edge_lines'] = calc_cross_pts(VA=vertexsA, VB=vertexsB, rngs=cases[idx]['ll_rng'])
    
    cases[idx]['name'] = fn.split('..')[0]
    
    ### resign values
    cases[idx]['ll_rng'] = [np.array(cases[idx]['edge_lines'][0]).min(),
                            np.array(cases[idx]['edge_lines'][0]).max(),
                            np.array(cases[idx]['edge_lines'][1]).min(),
                            np.array(cases[idx]['edge_lines'][1]).max()]
    cases[idx]['slice_pts'] = np.array(vslices)
    cases[idx]['symax'] = slice_ymax
    
    cases[idx]['slice_coords'], cases[idx]['TRMM_slice'], cases[idx]['terrain'] = generat_slice(cases[idx]['TRMM_profile'], 
                                                                         cases[idx]['X0'].ravel(), 
                                                                         cases[idx]['Y0'].ravel(), 
                                                                         cases[idx]['slice_pts'])
    _, cases[idx]['GPM_slice'], _ = generat_slice(cases[idx]['GPM_profile'], 
                                                                         cases[idx]['X1'].ravel(), 
                                                                         cases[idx]['Y1'].ravel(), 
                                                                         cases[idx]['slice_pts'],
                                                 is_terr=False)
    
    del handle
del ANA_handler

In [6]:
## scatter
loc_main = r'E:\data\_results\common_radar\used_data'

fn_20dBZ_Htop = os.path.join(loc_main, '_TG_2D_flag.csv')

base_legend = ['TRMM_PR', 'GPM_KuPR']

sfcType_sep = ['LandOcean_GPMKu']
sfcType = {0: 'Ocean', 1: 'Land'} # , 2: 'Coast'

rainType = {1: 'Stratiform', 2: 'Convective'}

self_colors = ['dodgerblue', 'salmon', 'limegreen', 'black']
linestyles = ['-', '--', ':', '-.']

properties = {'dBZ20_Htop': ['dBZ20_Htop (km)', 0.25, [0,20], 
                            ['dBZ20_Htop_TRMMKu', 'dBZ20_Htop_GPMKu', 'DLT_dBZ20_Htop', 'LandOcean_GPMKu'], 
                             1, [-10, 10]], 
             'dBZ35_Htop': ['dBZ35_Htop (km)', 0.25, [0,20], 
                            ['dBZ35_Htop_TRMMKu', 'dBZ35_Htop_GPMKu', 'DLT_dBZ35_Htop', 'LandOcean_GPMKu'], 
                            1, [-10, 10]], 
             'nearSurfdBZ': ['nearSurfdBZ (dBZ)', 1, [10,65], 
                             ['nearSurfdBZ_TRMMKu', 'nearSurfdBZ_GPMKu', 'DLT_nearSurfdBZ', 'LandOcean_GPMKu'], 
                             2, [-35, 35]],
             'nearSurfRain': [r'nearSurfRain (10$^x$ mm h$^{-1}$)', 0.1, [-1,3], 
                             ['nearSurfRain_TRMMKu', 'nearSurfRain_GPMKu', 'DLT_nearSurfRain', 'LandOcean_GPMKu'], 
                              0.1, [-3, 3]]}
thh_dBZ_GPM, thh_dBZ_TRMM = 18, 18  # dBZ

# constant defination of 1to1 comparison
bins_coincide = np.arange(0, 20.01, 0.25)

levs_coincide = np.array([1, 5 ]+list(np.arange(10,101, 5)))  # percentage
cb_labs = [1] + list(np.arange(10,101, 10))

tick_loc = [3,1,3,1]

In [7]:
df_dBZ20 = pd.read_csv(fn_20dBZ_Htop, compression='gzip')
df_dBZ20 = df_dBZ20.query(f'(dBZ20_Htop_TRMMKu > 0) & (dBZ20_Htop_GPMKu > 0) & (nearSurfdBZ_TRMMKu>={thh_dBZ_TRMM}) & (nearSurfdBZ_GPMKu>={thh_dBZ_GPM})')
df_dBZ20.columns, df_dBZ20.shape

(Index(['Time_TRMMKu', 'Time_GPMKu', 'dlt_Dm', 'dlt_Tsec', 'orbitID_TRMMKu',
        'loc_x_TRMMKu', 'loc_y_TRMMKu', 'orbitID_GPMKu', 'loc_x_GPMKu',
        'loc_y_GPMKu', 'Lon_TRMMKu', 'Lat_TRMMKu', 'LocalHour_TRMMKu',
        'nearSurfRain_TRMMKu', 'dBZ20_Htop_TRMMKu', 'dBZ35_Htop_TRMMKu',
        'rainType_TRMMKu', 'HBB_TRMMKu', 'Lon_GPMKu', 'Lat_GPMKu',
        'LocalHour_GPMKu', 'nearSurfRain_GPMKu', 'dBZ20_Htop_GPMKu',
        'dBZ35_Htop_GPMKu', 'LandOcean_GPMKu', 'rainType_GPMKu', 'HBB_GPMKu',
        'nearSurfdBZ_TRMMKu', 'surfRainPhase_TRMMKu', 'nearSurfdBZ_GPMKu',
        'surfRainPhase_GPMKu', 'dBZ20_Npxl_TRMMKu', 'dBZ35_Npxl_TRMMKu',
        'dBZ20_Npxl_GPMKu', 'dBZ35_Npxl_GPMKu', 'profile_TRMM', 'profile_GPM',
        'GmT_Yloc_M', 'GmT_Yloc_L', 'GmT_Yloc_R', 'nearSurfRain_TRMMKu_log10',
        'nearSurfRain_GPMKu_log10'],
       dtype='object'),
 (97619, 42))

In [8]:
# long-term
loc_raw = r'E:\data\_results\common_radar\used_data'
loc_res = r'E:\data\_results\records'

cols_all = {'Year': np.int32, 'Month': np.int32, 'DayOfMonth': np.int32, 'LocalHour': np.float32, 
        'Longitude': np.float32, 'Latitude': np.float32, 
        'LandOcean': np.int32, 'rainType': np.int32, 
        'ku_dBZ20_Htop': np.float32, 'ku_dBZ35_Htop': np.float32, 'nearSurfZ': np.float32, 'nearSurfRain': np.float32
       }

cols = ['Year', 'Month', 'Longitude', 'Latitude',  #'DayOfMonth', 
        'LandOcean', 'rainType', 'ku_dBZ20_Htop', 'ku_dBZ35_Htop', 'nearSurfZ'] #

fn_TRMM = os.path.join(loc_raw, 'TRMM_Rainfall_pixels_ku_dBZ_Htop_1998_2014.h5')
fn_GPM = os.path.join(loc_raw, 'GPM_Rainfall_pixels_ku_dBZ_Htop_2014_2020.h5')

rng_lat = [-36.2, 36.2]
lims_lat = [[-36.2, -20], [-20, 0], [0, 20], [20, 36.2]]


In [9]:
thh_dBZ_GPM, thh_dBZ_TRMM = 13, 18  # dBZ
def records(fn, df):
    sfn = os.path.join(loc_res, fn+'.csv')
    df.to_csv(sfn, header=True, index=False)
        
    return


def vaex_mean(df_Base, var, fn, grp=['Year', 'Month']):
    df = df_Base.groupby(by=grp,
               agg = {'Land_mean': ve.agg.mean(var, selection='LandOcean==1'),                      
                      'Ocean_mean': ve.agg.mean(var, selection='LandOcean==0'),
                      'Land_count': ve.agg.count(var, selection='LandOcean==1'),
                      'Ocean_count': ve.agg.count(var, selection='LandOcean==0'),
               }).sort(by=grp).to_pandas_df()
    records(fn, df)
    return df 


def vaex_mean_std(df_Base, var, fn, grp=['Year', 'Month']):
    df = df_Base.groupby(by=grp,
               agg = {'Land_mean': ve.agg.mean(var, selection='LandOcean==1'),
                      'Ocean_mean': ve.agg.mean(var, selection='LandOcean==0'),
                      'Land_std': ve.agg.std(var, selection='LandOcean==1'),
                      'Ocean_std': ve.agg.std(var, selection='LandOcean==0'),
                      'Land_count': ve.agg.count(var, selection='LandOcean==1'),
                      'Ocean_count': ve.agg.count(var, selection='LandOcean==0'),
               }).sort(by=grp).to_pandas_df()
    records(fn, df)
    return df 

suffix = 'whole'

In [41]:
vedfGPM = ve.open(fn_GPM)[cols]
vedfGPM_base = vedfGPM[  (vedfGPM.Latitude >= rng_lat[0]) & (vedfGPM.Latitude <= rng_lat[1])      # within Latitude ranges
              & (vedfGPM.Year < 2021)           # filter Y2021
            & (vedfGPM.ku_dBZ20_Htop > 0)   # must > 0 km
              & (vedfGPM.nearSurfZ>=thh_dBZ_GPM)
                      ]      

In [42]:
grp = ['Year', 'Month']
fn = 'TimeSeries_dBZ20_Htop_GPM_2014-2020'+grp[-1]+'_'+suffix
dfGPM20_M = vaex_mean(vedfGPM_base, 'ku_dBZ20_Htop', fn, grp=grp)

In [44]:
vedfTRMM = ve.open(fn_TRMM)[cols]
vedfTRMM_base = vedfTRMM[  (vedfTRMM.Latitude >= rng_lat[0]) & (vedfTRMM.Latitude <= rng_lat[1])      # within Latitude ranges
              & (vedfTRMM.Year > 1997)           # filter Y1997
            & (vedfTRMM.ku_dBZ20_Htop > 0)   # must > 0 km
              & (vedfTRMM.nearSurfZ>=thh_dBZ_TRMM)
                        ]

In [45]:
grp = ['Year', 'Month']
fn = 'TimeSeries_dBZ20_Htop_TRMM_1998-2014'+grp[-1]+'_'+suffix
dfTRMM20_M = vaex_mean(vedfTRMM_base, 'ku_dBZ20_Htop', fn, grp=grp)
dfTRMM20_M.drop(dfTRMM20_M.index.max(), inplace=True) # the last month has only 7 days, eliminated

In [10]:
# load old data
dfTRMM20_M = pd.read_csv(os.path.join(loc_res, 'TimeSeries_dBZ20_Htop_TRMM_1998-2014Month_all.csv'))
dfTRMM20_M.drop(dfTRMM20_M.index.max(), inplace=True) # the last month has only 7 days, eliminated
dfGPM20_M = pd.read_csv(os.path.join(loc_res, 'TimeSeries_dBZ20_Htop_GPM_2014-2020Month_all.csv'))

In [11]:
df_M = dfTRMM20_M.merge(dfGPM20_M, how='outer', on=['Year', 'Month'], sort=True, suffixes=base_legend)

time_rngs = ('19980101', '20201231')
YYYYMM = pd.date_range(start=time_rngs[0], end=time_rngs[1], freq='MS').to_series()
YYYYMM.name='Time'
xlab = pd.concat([YYYYMM, YYYYMM.apply(lambda x: x.strftime('%Y-%m')).str.split('-', expand=True).astype(int)], axis=1)

df_M = df_M.merge(xlab, how='right', left_on=['Year', 'Month'], right_on=[0, 1])
df_M.drop(columns=[0,1,'Year', 'Month'], inplace=True)

xrange_M, yrange_M = [df_M.Time.min(), df_M.Time.max()], [3, 8]

df_M.shape

(276, 33)

In [12]:
grid_ll = 1
TRMM_mean_annual = {}
fn = 'dBZ20_Htop_LongTerm_1998-2020_res'+f'{grid_ll:.2f}'+'_all.h5'
sfn = os.path.join(loc_res, fn)
datmp = h5py.File(sfn, 'r')

dlt_Htop = datmp['dlt_dBZ20_Htop'][:]
#print(datmp['count_dBZ20_Htop_TRMM'][:].min(), datmp['count_dBZ20_Htop_GPM'][:].min(),
#      dlt_Htop.min(), dlt_Htop.max())

lons = datmp['Lons'][:]
lats = datmp['Lats'][:]
mean_Htop = datmp['mean_dBZ20_Htop'][:]
mean_TRMM = datmp[ 'mean_dBZ20_Htop_TRMM'][:]
mean_GPM = datmp['mean_dBZ20_Htop_GPM'][:]
count_TRMM = datmp['count_dBZ20_Htop_TRMM'][:]
count_GPM = datmp['count_dBZ20_Htop_GPM'][:]
TRMM_mean_annual['std'] = datmp['std'][:]

datmp.close()

In [13]:
lon_center = 0
cb_shrink, cb_orientation, aspect = 0.9, 'vertical', 15
chr_fontsize=10

pts_colors = ['b', 'r']
DD = pd.Timedelta(365, unit='day')

abstract = 'The 20'+r'$-$'+'dBZ echo'+r'$-$'+'top heights are consistent in different spatiotemporal scales\n'+ \
           "between Ku"+r'$-$'+"band radars onboard TRMM and GPM."


from sklearn.metrics import mean_squared_error
import matplotlib.dates as mdates
#############
def plot_GA(Xs, Ys, data, name, units=[],save_zero=True, ll_rng=[], levs=[], edge_lines=[],
                  titles=[], cb_labels=[],
                  cmap_rng=list(np.arange(6)+1), extend='max', ckind='pr1', pic_subdir='',
                  L_slice=[], C_slice=[], V_slice=[], terrain=[], Symax=20,
                  sfx_text_loc = [[0.01, 0.98, 'left'], [[0, 0,'top'],[0, 0,'top']]] ):
    
    plot = Plotting(global_xy=True, fontsize=12)
    
    cmap1 = plot.icmap.set_cmap(rng=cmap_rng, c_firstwhite=False, kind=ckind) 
#    cmap2 = copy.copy((cm.get_cmap("bwr")))
    
    sns.set_context('paper', font_scale=1.4)  #         
    fig, _ = plt.subplots(figsize=(11, 6))
    fig.subplots_adjust(wspace=0.28, hspace=0.18)
    _.remove()
    gs = fig.add_gridspec(ncols=1, nrows=2) #, height_ratios=[4, 2.2, 2.2]
#    print(ax.shape)
    '''
    ####
    ax1 = fig.add_subplot(gs[0, 0])
    ax1.axis('off')
    
    #======
    iax = ax1.inset_axes(bounds=[0,0.5,1,0.5])
    Yc, Xc = np.meshgrid(hgt, C_slice[0])
    obj = iax.contourf(Xc-lon_center, Yc, V_slice[0], 
                          cmap=cmap1, levels=levs[0], 
                          norm=colors.BoundaryNorm(boundaries=np.array(levs[0]), ncolors=cmap1.N), 
                          alpha=1, zorder=1, extend=extend)
    obj.cmap.set_over('indigo')

    iax.fill_between(C_slice[0], np.zeros_like(terrain),terrain, color='k') #, transform=proj

    iax.set_ylim(0, Symax) #hgt.max()
    iax.yaxis.set_major_locator(ticker.MultipleLocator(3))
#    iax.yaxis.set_minor_locator(ticker.MultipleLocator(1))
#    iax.tick_params(axis='y', which='major', labelsize=12)
    iax.text(x=0.02, y=0.95, s='TRMM_PR', fontsize=chr_fontsize, 
             ma='left', ha='left', va='top', color='black', transform=iax.transAxes)
    
    #==============
    iax = ax1.inset_axes(bounds=[0,0,1,0.5])
    Yc, Xc = np.meshgrid(hgt, C_slice[0])
    obj = iax.contourf(Xc-lon_center, Yc, V_slice[1], 
                          cmap=cmap1, levels=levs[0], 
                          norm=colors.BoundaryNorm(boundaries=np.array(levs[0]), ncolors=cmap1.N), 
                          alpha=1, zorder=1, extend=extend)
    obj.cmap.set_over('indigo')

    iax.fill_between(C_slice[0], np.zeros_like(terrain),terrain, color='k') #, transform=proj
#        iax.set_xlim(L_slice[0][0], L_slice[1][0])

    iax.set_ylim(0, Symax) #hgt.max()
    iax.yaxis.set_major_locator(ticker.MultipleLocator(3))
#    iax.yaxis.set_minor_locator(ticker.MultipleLocator(1))
#    iax.tick_params(axis='y', which='major', labelsize=12)
    iax.invert_yaxis()
    iax.text(x=0.02, y=0.04, s='GPM_KuPR', fontsize=chr_fontsize, 
             ma='left', ha='left', va='bottom', color='black', transform=iax.transAxes)

    ax1.text(x=-0.12, y=0.5, s='Profile height (km)', rotation=90, 
            ma='left', ha='center', va='center', color='black', transform=ax1.transAxes)
    
    cb_ax = ax1.inset_axes(bounds=[1.03, 0.0, 0.035, 0.9])
    cb = fig.colorbar(obj, cb_ax, orientation=cb_orientation) #shrink=cb_shrink, aspect=15, pad=0,  

    cb.ax.tick_params(direction='in', length=4)
    cb.ax.tick_params(which='minor', direction='in', length=2)
    tick_labs = plot.__shrink_tail_zeros__(cb.get_ticks())
#    cb.ax.set_yticklabels(tick_labs, fontdict={'fontsize': plot.fontsize})  
#    cb.set_label(' '.join(['Reflectivity', '('+units[0]+')']), labelpad=1) #fontdict={'fontsize': plot.fontsize}, 
    ax1.text(x=1.08, y=0.93, s='(dBZ)', 
            ma='left', ha='center', va='bottom', color='black', transform=ax1.transAxes)
    ###########
    ax2 = fig.add_subplot(gs[0, 1])
    Vars = ['dBZ20_Htop_TRMMKu', 'dBZ20_Htop_GPMKu']
    data = df_dBZ20
    grid, Xedge, Yedge, _ = ax2.hist2d(data[Vars[0]], data[Vars[1]], 
                                       bins=bins_coincide, visible=False)

    totDA, maxDA = data.shape[0], int(grid.max())
    pctDA = 100*grid/maxDA

    obj = ax2.pcolormesh(Xedge, Yedge, pctDA,
                  cmap=cmap1, shading='auto', 
                  norm=colors.BoundaryNorm(boundaries=levs_coincide, ncolors=cmap1.N), 
                  edgecolors='none', alpha=1, zorder=2)
    obj.cmap.set_under('white')
    obj.cmap.set_over('firebrick')

    ax2.axline(xy1=(bins.min(), bins.min()), xy2=(bins.max(), bins.max()),
                       linestyle='--', color='k')
    
    pcc = data[Vars].corr().to_numpy()[0, 1]
    rmse = np.sqrt(mean_squared_error(data[Vars[0]], data[Vars[1]]))
    bias = (data[Vars[1]]-data[Vars[0]]).mean()

    ax2.text(x=0.02, y=0.98, 
             s=f'PCC = {pcc:4.02f}\nBIAS = {bias:4.02f}\nRMSE = {rmse:4.02f}', fontsize=chr_fontsize, 
             ma='left', ha='left', va='top', color='black', transform=ax2.transAxes) #, fontsize=fz

    ax2.xaxis.set_major_locator(ticker.MultipleLocator(tick_loc[0]))
    ax2.xaxis.set_minor_locator(ticker.MultipleLocator(tick_loc[1]))
    ax2.yaxis.set_major_locator(ticker.MultipleLocator(tick_loc[2]))
    ax2.yaxis.set_minor_locator(ticker.MultipleLocator(tick_loc[3]))
    
    ax2.set_xlim([0,16])
    ax2.set_ylim([0,16])

#    ax2.set_ylabel(f'{base_legend[0]} (km)')
#    ax2.set_xlabel(f'{base_legend[1]} (km)')
    
    ax2.text(x=0.5, y=0.06, s=f'{base_legend[1]} (km)', fontsize=chr_fontsize, 
             ma='left', ha='center', va='top', color='black', transform=ax2.transAxes)
    ax2.text(x=0.06, y=0.5, s=f'{base_legend[0]} (km)', fontsize=chr_fontsize, rotation=90, 
             ma='left', ha='right', va='center', color='black', transform=ax2.transAxes)

    cb_ax = ax2.inset_axes(bounds=[1.03, 0., 0.035, 0.9])
    cb = fig.colorbar(obj, cb_ax, orientation='vertical') #shrink=cb_shrink, aspect=15, pad=0,  

    cb.set_ticks(cb_labs)
    cb.set_ticklabels(cb_labs)
    cb.ax.tick_params(direction='in', length=4)
    cb.ax.tick_params(which='minor', direction='in', length=2)
#            tick_labs = plot.__shrink_tail_zeros__(cb.get_ticks())
#            cb.ax.set_yticklabels(tick_labs, fontdict={'fontsize': plot.fontsize})  
#    cb.set_label(' Percentage (%)', labelpad=1) #, fontdict={'fontsize': plot.fontsize}
    ax2.text(x=1.07, y=0.93, s='(%)', 
            ma='left', ha='center', va='bottom', color='black', transform=ax2.transAxes)
    '''
    ########
    ax3 = fig.add_subplot(gs[0])
    yrange_M=[3, 8]
    
    for vdx, var in enumerate(base_legend):
        ax3.plot(df_M.Time, df_M['Ocean_mean_NH20'+var], 
                     color=self_colors[vdx], linestyle=linestyles[0], zorder=2, #linewidth=4, 
                     label='_'.join([var, 'Ocean']))
        ax3.plot(df_M.Time, df_M['Land_mean_NH20'+var], 
                     color=self_colors[vdx], linestyle=linestyles[1], zorder=2, #linewidth=4, 
                     label='_'.join([var, 'Land']))
    
    ax3.yaxis.set_major_locator(ticker.MultipleLocator(1))
    ax3.yaxis.set_minor_locator(ticker.MultipleLocator(0.25))
    ax3.set_ylim(yrange_M)
    ax3.set_ylabel('Height (km)')
        
    ax3.set_xlim(xrange_M)        
    ax3.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
    ax3.xaxis.set_major_locator(mdates.YearLocator(4))
    ax3.xaxis.set_minor_locator(mdates.YearLocator(1))    
    
    hands, labels = ax3.get_legend_handles_labels()
    ax3.legend(hands, labels, loc='best', ncol=4, fontsize=chr_fontsize)
        
    #######
    proj = ccrs.PlateCarree(central_longitude=lon_center)
    ax4 = fig.add_subplot(gs[1], projection=proj)
    
    levs0 = np.arange(1, 9, 0.2).tolist()
    ll_rng = [-180, 180+1.e-8, -36.5, 36.5+1.e-8]
    
    obj0 = ax4.contourf(lons-lon_center, lats, mean_Htop+1.e-8, 
                          cmap=cmap1, levels=levs0, 
                          norm=colors.BoundaryNorm(boundaries=np.array(levs0), ncolors=cmap1.N), 
                          alpha=1, zorder=2, extend='max')
    obj0.cmap.set_over('indigo')
    gl, cb = plot.__load_ax_settings__(fig, ax4, proj, obj0, var='', rng=ll_rng, unit='km', lon_center=lon_center,
                             pad=0.025, aspect=0)
    
    ax4.gridlines(draw_labels=False, linewidth=2, linestyle=':', color='gray', alpha=0.8)
    
    colorbar_ax = ax4.inset_axes(bounds=[1.01, 0, 0.015, 1])
    cb = fig.colorbar(obj0, colorbar_ax, aspect=40, #label='Sample number',
                      orientation='vertical')    
        
    cb.ax.tick_params(which='major', length=3)
#    cb.ax.tick_params(which='minor', length=4)
    tick_labs = np.arange(1,9,1).astype(int)
    cb.ax.yaxis.set_ticks(tick_labs)
    cb.ax.set_yticklabels(tick_labs)
#    cb.ax.set_ylabel('Height (km)', color='k') #, fontsize=14
    ax4.text(x=1.02, y=1.01, s='(km)', 
            ma='left', ha='center', va='bottom', color='black', transform=ax4.transAxes)
    
    iax = ax3
    iax.text(x=1, y=1.02, s=abstract, 
            ma='right', ha='right', va='bottom', color='black', transform=iax.transAxes)
    plot._save_fig(name, fig)
    del plot 
    return

In [14]:
# in relation to vslice
sfx_text_locs = [[0.99, 0.98, 'right'], [[0.3, -0.05,'bottom'],[0, 0, 'bottom']]]

##############
case = cases[0]
cc0 = (case['X0']>=case['ll_rng'][0]) * (case['X0']<=case['ll_rng'][1]) *(case['Y0']>=case['ll_rng'][2])*(case['Y0']<=case['ll_rng'][3])
cc1 = (case['X1']>=case['ll_rng'][0]) * (case['X1']<=case['ll_rng'][1]) *(case['Y1']>=case['ll_rng'][2])*(case['Y1']<=case['ll_rng'][3])
time_idx = int(np.where(cc0.sum(axis=1) > 23)[0].mean()), int(np.where(cc1.sum(axis=1) > 23)[0].mean())

plot_GA([case['X0'], case['X1']], [case['Y0'], case['Y1']], 
              [[case['TRMM_nearSurfdBZ'],case['GPM_nearSurfdBZ']],[case['TRMM_nearSurfRain']+1.e-8,case['GPM_nearSurfRain']+1.e-8]],
              'GA', 
              levs=[case['levs_dBZ'], case['levs_RR']], units=[case['unit_dBZ'], case['unit_RR']],
              ll_rng=case['ll_rng'], edge_lines = case['edge_lines'],
              cb_labels=['nearSurfdBZ', 'nearSurfRain'],
              titles=[f"TRMM Orbit: {case['TRMM_Orbit']}    "+str(case['TRMM_time'][time_idx[0]])[2:-1], 
                      f"GPM Orbit: {case['GPM_Orbit']}          "+str(case['GPM_time'][time_idx[1]])[2:-1]],
              L_slice = case['slice_pts'],
              C_slice = case['slice_coords'],
              V_slice = [case['TRMM_slice'], case['GPM_slice']],
              terrain = case['terrain'],
              Symax = case['symax']
             ) 
#    break
